In [ ]:
import preprocessor as p
import numpy as np

In [ ]:
%run ../twitter16/twitter16_text_processing.ipynb

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
import pickle as pkl
from collections import defaultdict
import pandas as pd
import os
import numpy as np
import json
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [ ]:
from collections import Counter
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
import random

In [ ]:
class Node:
    def __init__(self,uid,tid,time_stamp,label):
        self.children = {}
        self.childrenList = []
        self.num_children = 0
        self.tid = tid
        self.uid = uid
        self.label = label
        self.time_stamp = time_stamp
    
    def add_child(self,node):
        if node.uid not in self.children:
            self.children[node.uid] = node
            self.num_children += 1
        else:
            self.children[node.uid] = node
        self.childrenList = list(self.children.values())

In [ ]:
class Tree:
    def __init__(self,root):
        self.root = root
        self.tweet_id = root.tid
        self.uid = root.uid
        self.height = 0
        self.nodes = 0
    
    def show(self):
        queue = [self.root,0]
        
        while len(queue) != 0:
            toprint = queue.pop(0)
            if toprint == 0:
                print('\n')
            else:
                print(toprint.uid,end=' ')
                queue += toprint.children.values()
                queue.append(0)
                
    def insertnode(self,curnode,parent,child):
        if curnode.uid == parent.uid:
            curnode.add_child(child)
            return 1

        elif parent.uid in curnode.children:
            s = self.insertnode(curnode.children[parent.uid],parent,child)
            return 2
        else:
            for node in curnode.children:
                s = self.insertnode(curnode.children[node],parent,child)
                if s == 2:
                    break

In [ ]:
def loadPklFileNum(datapath,incSize,fileNum):
    
    with open(datapath+str(incSize)+'inc_'+str(fileNum)+'.pickle', 'rb') as handle:
        twitTrees = pkl.load(handle)
    return twitTrees

In [ ]:
def loadTreeFilesOfIncrement(datapath,incSize):
    twittertrees = {}
    
    files = [x for x in os.listdir(t15Datapath) if str(incSize)+'inc' in x]
    
    for file in tqdm(files):
        with open(datapath+file,'rb') as handle:
            partialTrees = pkl.load(handle)
        twittertrees.update(partialTrees)
        
    return twittertrees

In [ ]:
t15Datapath = '../twitter16/pickledTrees/'
# twitter15_trees = loadPklFileNum(t15Datapath,20,1)

In [ ]:
twitter15_trees = loadTreeFilesOfIncrement(t15Datapath,20)

In [ ]:
%run ../twitter16/userdata_parser.ipynb

In [ ]:
for key in tqdm(userVects):
    userVects[key] = userVects[key].float()

userVects = defaultdict(lambda:torch.tensor([1.1100e+02, 1.5000e+01, 0.0000e+00, 7.9700e+02, 4.7300e+02, 0.0000e+00,
        8.3326e+04, 1.0000e+00]),userVects)

In [ ]:
%run ./textEncoders.ipynb

In [ ]:
%run ./temporal_tree_model.ipynb

In [ ]:
if torch.cuda.is_available():
    device = 'cuda:1'
    device = 'cpu'
else:
    device = 'cpu'

In [ ]:
labelMap = {}
labelCount = 0
for label in list(twitter15_labels.values()):
    if label not in labelMap:
        labelMap[label] = labelCount
        labelCount += 1
labelMap

In [ ]:
epochs = 10
X = []
y = []
X_text = []

for tid in twitter15_trees:
        if tid in twitter15_trees and tid in twitter15_labels:
            X.append(tuple((twitter15_trees[tid],twitter15_text[tid])))
            y.append(labelMap[twitter15_labels[tid]])
            X_text.append(twitter15_text[tid])
            
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=2018)

In [ ]:
textparams = {
    'embedding_dim':256,
    'hidden_dim': 50,
    'output_dim':4,
    'num_layers':1,
    'bidir':True,
    'rnnType':'gru'
}

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
model = temporalDecayTreeEncoder(torch.cuda.is_available(),8,100,userVects,twitter15_labels,labelMap,criterion,device)
model = model.to(device)

In [4]:
f = lambda m, n: [(i*n//m + n//(2*m)) for i in range(m)]

In [1]:
optimizer = torch.optim.Adagrad(model.parameters(),lr=0.01)

count = 0
maxAcc = 0

train_iterwise = []
val_iterwise = []

for i in range(7):
    train_losses = []
    val_losses = []
    
    for treeSet,text in tqdm(x_train):
        tnum = 0
        
        idxs = f(5,len(treeSet))
#         for idx in idxs:
        trees = [ treeSet[idx] for idx in idxs ]
        count += 1
        tnum += 1
        optimizer.zero_grad()

        pred = model(trees)
        label = Variable(torch.tensor(labelMap[trees[0].root.label]))

        loss = criterion(pred.reshape(-1,4), label.reshape(-1))    

        loss.backward()
        optimizer.step()
            
    preds = []
    labels = []

    allLabels = []
    allPreds = []

    with torch.no_grad():
        for valSet,text in tqdm(x_test):
            idxs = f(5,len(valSet))
            trees = [ valSet[idx] for idx in idxs ]
            
            predicted = model(trees)
            preds.append(predicted)

            predicted =  torch.softmax(predicted[0],0)
            predicted = torch.max(predicted, 0)[1].cpu().numpy().tolist()

            labels.append(labelMap[trees[0].root.label])

            allLabels.append(labelMap[trees[0].root.label])
            allPreds.append(predicted)

    predTensor = torch.stack(preds)
    labelTensor = torch.tensor(labels).to(device)

    print(allLabels,allPreds)

    loss = criterion(predTensor.reshape(-1,4), labelTensor.reshape(-1))

    cr = classification_report(allLabels,allPreds,output_dict=True)
    cr['loss'] = loss.item()
    cr['Acc'] = accuracy_score(allLabels,allPreds,)
    print('loss: ',cr['loss'])
    print(cr['Acc'])
    
    if cr['Acc'] > maxAcc:
            maxAcc = cr['Acc']
            torch.save({'state_dict': model.state_dict()}, './pretrainedModels-Twit16/'+'decay_tempTreeEnc_pretrained_inc5'+'.pth')
    
    with open('./pretrainedModels-Twit16/'+'decay_tempTreeEnc_pretrained_inc5'+'.json', 'a') as fp:
            json.dump(cr, fp)
            fp.write('\n')
        
    val_losses.append(loss.item())
    train_iterwise.append(np.array(train_losses).mean())
    val_iterwise.append(np.array(val_losses).mean())

NameError: name 'torch' is not defined

rnnTypes = ['gru','lstm']
bidirTypes = [True,False]

for rnnType in rnnTypes:
    for bidirType in bidirTypes:
        textparams[bidir] = bidirType
        textparams[rnnType] = rnnType
        
        textEncoderModel = TextEncoder('rnn',textparams,X_text,y,device)
        textEncoderModel.trainModel()
        print(len(textEncoderModel.word2idx))
        
        modelname = rnnType
        if bidirType:
            modelname = 'bidir'+modelname
        
        torch.save({'state_dict': textEncoderModel.optimalParams}, './pretrainedModels-Twit15/'+modelname+'.pth')